<a href="https://colab.research.google.com/github/Rbeautelus1/Rbeautelus1/blob/main/BERTA%20V%20ROBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import re
import string
!pip install emoji
import emoji

from transformers import RobertaTokenizer
from datasets import Dataset

import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split

from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Loading Dataset

In [3]:
data1 = pd.read_csv("/content/Tweets.csv")
data2 = pd.read_csv("/content/stock_data.csv")




In [4]:
data1.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
data2.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


Clean dataset 1

In [6]:
import re
import string
import emoji
import spacy
import nltk
from nltk.corpus import stopwords

# Download the stopwords dataset
nltk.download('stopwords')

# Load the spaCy model once
nlp = spacy.load('en_core_web_sm')

# Load stopwords
stopwords_list = set(stopwords.words('english'))

# Function to remove emojis from text
def strip_emoji(text):
    return emoji.replace_emoji(text, replace='')

# Function to strip all entities (newline, emojis, mentions, links, non-ASCII characters)
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()  # Remove newlines and lowercase
    text = re.sub(r'[^\x00-\x7f]', r'', text)  # Remove non-ASCII characters
    banned_list = string.punctuation + 'Ã' + '±' + 'ã' + '¼' + 'â' + '»' + '§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)  # Remove banned characters
    return text

# Function to clean hashtags
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet))
    return " ".join(word.strip() for word in re.split('#|_', new_tweet))  # Remove hashtags in the middle

# Main function to apply all cleaning steps
def text_cleaning(text):
    # Lowercase the text and remove emojis
    text = strip_emoji(text.lower())

    # Remove mentions and URLs
    text = re.sub(r'@\w+', '', text)  # Remove @username
    text = re.sub(r'https?://\S+', '', text)  # Remove URLs

    # Remove digits and special characters
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters

    # Remove emojis, hashtags, and non-ASCII characters
    text = strip_all_entities(text)
    text = clean_hashtags(text)

    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords_list)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Ensure consistent column names for both datasets
data1.rename(columns={'text': 'Text'}, inplace=True)
data2.rename(columns={'Text': 'Text'}, inplace=True)

data1.rename(columns={'airline_sentiment': 'Sentiment'}, inplace=True)
data2.rename(columns={'Sentiment': 'Sentiment'}, inplace=True)

# Apply text cleaning to your datasets
data1['Cleaned_text'] = data1['Text'].apply(text_cleaning)
data1['Cleaned_text'] = data1['Cleaned_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

data2['Cleaned_text'] = data2['Text'].apply(text_cleaning)
data2['Cleaned_text'] = data2['Cleaned_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Check the result for both datasets
print("Cleaned data1 texts:")
for text in data1['Cleaned_text'].iloc[0:10]:  # Display the first 10 cleaned texts for data1
    print(text)

print("\nCleaned data2 texts:")
for text in data2['Cleaned_text'].iloc[0:10]:  # Display the first 10 cleaned texts for data2
    print(text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cleaned data1 texts:
said
plus youve added commercials experience tacky
didnt today must mean need take another trip
really aggressive blast obnoxious entertainment guests faces amp little recourse
really big bad thing
seriously would pay flight seats didnt playing really bad thing flying va
yes nearly every time fly vx ear worm wont go away
really missed prime opportunity men without hats parody
well didntbut
amazing arrived hour early youre good

Cleaned data2 texts:
kickers watchlist xide tit soq pnk cpw bpz aj trade method method see prev posts
user aap movie return feageed indicator trades year awesome
user id afraid short amzn looking like nearmonopoly ebooks infrastructureasaservice
mnta
oi
pgnx
aap user current downtrend break otherwise shortterm correction medterm downtrend
mondays relative weakness nyx win tie tap ice int bmc aon c chk biib
goog ower trend line channel test volume support
aap watch tomorrow ong entry


In [7]:
data1.head()

,tweet_id,Sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,Text,tweet_coord,tweet_created,tweet_location,user_timezone,Cleaned_text
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),plus youve added commercials experience tacky
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),didnt today must mean need take another trip
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),really aggressive blast obnoxious entertainmen...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),really big bad thing


In [8]:
data2.head()

,Text,Sentiment,Cleaned_text
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,kickers watchlist xide tit soq pnk cpw bpz aj ...
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,user aap movie return feageed indicator trades...
2,user I'd be afraid to short AMZN - they are lo...,1,user id afraid short amzn looking like nearmon...
3,MNTA Over 12.00,1,mnta
4,OI Over 21.37,1,oi


Merging Dataset

In [9]:
# Keep only the necessary columns for sentiment analysis
data1_cleaned = data1[['Cleaned_text', 'Sentiment']]
data2_cleaned = data2[['Cleaned_text', 'Sentiment']]

In [10]:
# Merge the two datasets
merged_data = pd.concat([data1_cleaned, data2_cleaned], ignore_index=True)

In [11]:
# Check the merged dataset
print(merged_data.head())

                                        Cleaned_text Sentiment
0                                               said   neutral
1      plus youve added commercials experience tacky  positive
2       didnt today must mean need take another trip   neutral
3  really aggressive blast obnoxious entertainmen...  negative
4                               really big bad thing  negative


In [12]:
# Check for missing values
missing_values = merged_data.isna().sum()
print("Missing Values:\n", missing_values)


Missing Values:
 Cleaned_text    0
Sentiment       0
dtype: int64


In [13]:
print(merged_data.dtypes)


Cleaned_text    object
Sentiment       object
dtype: object


In [14]:
print(merged_data['Sentiment'].value_counts())


Sentiment
negative    9178
1           3685
neutral     3099
positive    2363
-1          2106
Name: count, dtype: int64


In [15]:
merged_data.to_csv('cleaned_data.csv', index=False)


In [16]:
loaded_data = pd.read_csv('cleaned_data.csv')
print(loaded_data.head())


                                        Cleaned_text Sentiment
0                                               said   neutral
1      plus youve added commercials experience tacky  positive
2       didnt today must mean need take another trip   neutral
3  really aggressive blast obnoxious entertainmen...  negative
4                               really big bad thing  negative


# ROBERTA Model

In [17]:
merged_data = pd.read_csv('cleaned_data.csv')

Prepare the Data

In [21]:
! pip install --upgrade datasets
from datasets import Dataset


import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Encode the sentiment labels
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}  # Adjust according to your labels
merged_data['label'] = merged_data['Sentiment'].map(label_map)

# Split into training and validation sets
train_df, val_df = train_test_split(merged_data, test_size=0.2, random_state=42)

# Create datasets
train_dataset = Dataset.from_pandas(train_df[['Cleaned_text', 'label']])
val_dataset = Dataset.from_pandas(val_df[['Cleaned_text', 'label']])

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00


Load the tokenizer and Model

In [23]:
print(train_dataset)
print(val_dataset)


Dataset({
    features: ['Cleaned_text', 'label', '__index_level_0__'],
    num_rows: 16344
})
Dataset({
    features: ['Cleaned_text', 'label', '__index_level_0__'],
    num_rows: 4087
})


In [26]:
print(train_dataset[0])  # Print the first entry to see its structure
print(val_dataset[0])    # Same for validation dataset


{'Cleaned_text': 'cbmx history repeated another hge day look bgmd tek exact pattern cbmx float mill tek', 'label': None, '__index_level_0__': 17559}
{'Cleaned_text': 'abbott laboratories takes ride coronavirus crisis happens make three', 'label': None, '__index_level_0__': 19631}


In [34]:
{'Cleaned_text': 'cbmx history repeated another hge day...', 'label': 0}


{'Cleaned_text': 'cbmx history repeated another hge day...', 'label': 0}

In [35]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['Cleaned_text'], padding='max_length', truncation=True)


In [37]:
from transformers import RobertaTokenizer
from datasets import Dataset
import pandas as pd

# Sample DataFrames
train_df = pd.DataFrame({
    'Cleaned_text': ['cbmx history repeated another hge day...', 'sample text two'],
    'label': [0, 1]
})
val_df = pd.DataFrame({
    'Cleaned_text': ['abbott laboratories takes ride...', 'sample text four'],
    'label': [0, 1]
})

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['Cleaned_text'], padding='max_length', truncation=True)

# Tokenize the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Rename label column to labels
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_val = tokenized_val.rename_column("label", "labels")

# Set the format of the datasets for PyTorch or TensorFlow
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Check the first entry of the tokenized train dataset
print(tokenized_train[0])



Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'labels': tensor(0), 'input_ids': tensor([    0,   438, 43531,  1178,   750,  6636,   277,  1368,  1899,   183,
          734,     2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1, 

Training the Model

In [38]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define your training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Start training
trainer.train()


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.7074700196584066, metrics={'train_runtime': 76.2975, 'train_samples_per_second': 0.079, 'train_steps_per_second': 0.039, 'total_flos': 1578666332160.0, 'train_loss': 0.7074700196584066, 'epoch': 3.0})

In [39]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.6945397853851318, 'eval_runtime': 6.8796, 'eval_samples_per_second': 0.291, 'eval_steps_per_second': 0.145, 'epoch': 3.0}


In [40]:
trainer.save_model("./final_model")
